In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.layers import LSTM
from keras.preprocessing import sequence
from keras.layers import Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


/kaggle/input/dataasssignment/train.csv
/kaggle/input/dataasssignment/test_without_labels.csv


In [2]:
train_dataset = pd.read_csv('/kaggle/input/dataasssignment/train.csv')
test_dataset = pd.read_csv('/kaggle/input/dataasssignment/test_without_labels.csv')

In [3]:
test_dataset.head()

,Id,Content
0,0,This has to be one of the best movies we have ...
1,1,"Over the last few months, I have seen a lot of..."
2,2,This movie was so awful i don't even know wher...
3,3,The worst Wrestlemania ever.<br /><br />This h...
4,4,I first saw this film in the theater way back ...


In [4]:
word_to_id = keras.datasets.imdb.get_word_index()

1646592/1641221 [==============================] - 0s 0us/step


In [5]:
train_strings = train_dataset['Content'].tolist()
test_strings = test_dataset['Content'].tolist()

Convert texts to vectors 

In [6]:


from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

for s in range(len(train_strings)):
    train_strings[s] = tokenizer.tokenize(train_strings[s].lower())
    for i in range(len(train_strings[s])):
        try:
            train_strings[s][i] = word_to_id[train_strings[s][i]]
        except:
            train_strings[s][i] = 0

for s in range(len(test_strings)):
    test_strings[s] = tokenizer.tokenize(test_strings[s].lower())
    for i in range(len(test_strings[s])):
        try:
            test_strings[s][i] = word_to_id[test_strings[s][i]]
        except:
            test_strings[s][i] = 0

In [7]:
x_train = sequence.pad_sequences(train_strings, maxlen=500)
x_test = sequence.pad_sequences(test_strings, maxlen=500)

In [8]:
len(word_to_id)

88584

In [9]:

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',mode='min', factor=0.2,
                              patience=1, min_lr=0,verbse=1, min_delta=0.0000001)

In [10]:

def create_model():
    model = Sequential()
    model.add(Embedding(88584, 64, input_length=500))
    model.add(Dropout(0.25))
    model.add(Conv1D(32,
                 5,
                 padding='valid',
                 activation='relu',
                 strides=1))
    model.add(MaxPooling1D(pool_size=4))
    model.add(LSTM(128, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer= keras.optimizers.Adam(learning_rate=0.0001),metrics=['accuracy'])
    return model

# try using different optimizers and different optimizer configs

#score, acc = model.evaluate(x_test, y_test, batch_size=32)
#print('Test score:', score)
#print('Test accuracy:', acc)

In [11]:
metrics = {}

Do 5-fold cross validation then fit on the whole train set and predict test set


In [12]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
skf = StratifiedKFold(n_splits=5)
i=0
for train_index, test_index in skf.split(x_train, train_dataset['Label']):
    print("training split :",i+1)
    model = create_model()
    model.fit(x_train[train_index],train_dataset['Label'][train_index],validation_data =(x_train[test_index],train_dataset['Label'][test_index]),batch_size=2048,epochs=50, verbose =0)
    predictions = model.predict(x_train[test_index])
    predictions = np.where(predictions > 0.5, 1, 0)
    model_metrics = precision_recall_fscore_support(y_true =train_dataset['Label'][test_index] , y_pred = predictions, average='macro')
    metrics[i] = [model_metrics, accuracy_score(y_true =train_dataset['Label'][test_index] , y_pred = predictions)]
    i+=1
    

training split : 1


/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


training split : 2


/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


training split : 3


/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


training split : 4


/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


training split : 5


/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [13]:
metrics

{0: [(0.8916141135883647, 0.891, 0.8909572508806353, None), 0.891],
 1: [(0.8928160897470361, 0.8928, 0.8927989022607592, None), 0.8928],
 2: [(0.8844711117768342, 0.8844000000000001, 0.8843946544088199, None),
  0.8844],
 3: [(0.8918226316752056, 0.8917999999999999, 0.8917984375694386, None),
  0.8918],
 4: [(0.9006231399925659, 0.9006000000000001, 0.9005985646432735, None),
  0.9006]}

In [14]:
from numpy import array
prec = array([metrics[k][0][0] for k in metrics]).mean()
rec = array([metrics[k][0][1] for k in metrics]).mean()
f1 = array([metrics[k][0][2] for k in metrics]).mean()
acc = array([metrics[k][1] for k in metrics]).mean()
print(prec,rec,f1,acc)

0.8922694173560014 0.89212 0.8921095619525854 0.89212


In [15]:
model = create_model()
model.fit(x_train,train_dataset['Label'],batch_size=2048,epochs=50, verbose =0)


/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [16]:
test_predictions = model.predict(x_test)

In [17]:
test_predictions = np.where(test_predictions > 0.5, 1, 0)

In [18]:
test_predictions

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [19]:
from pandas.core.common import flatten
#test_predictions = np.where(test_predictions > 0.5, 1, 0)
test_predictions = flatten(test_predictions)

prediction = pd.DataFrame(data={"Predicted": test_predictions}, index=test_dataset['Id'])
prediction.to_csv('testSet_categories.csv')